# Digital Marketing Campaigns Performance Analysis

This project aims to automate the report creation process and simplify the analysis of digital marketing ads on various platforms, including Google, Facebook, Twitter, LinkedIn, and Bing. The analysis is based on campaign performance over a specific period, usually exceeding three months, and examines monthly performance within that period.

The project uses a linear regression model to determine the trend of digital marketing campaigns. The model analyzes the monthly data for three key performance metrics, namely: 
- Return on Ad Spend (ROAS)
- Cost per Acquisition (CPA)
- Conversion Rate from Lead to Purchase


I chose to use linear regression on the monthly data for the key metrics because they exhibited a relatively smooth trend. Additionally, the scoring system that was created in the code ensured that the data points for each metric were more or less on the same level (more on this is explained below). Because of this, linear regression was a suitable choice for analyzing the trend and determining its direction. If the data had been noisy, I would have applied a moving average to remove the noise and make the trend more visible. However, in this case, the consistent linear pattern made linear regression a more appropriate choice.

By studying the trend obtained from the linear regression, the project can identify whether campaign performance has improved or declined over the given period. This data is critical in assessing the effectiveness of the campaigns and making data-driven decisions for future marketing strategies.

In addition to the linear regression analysis, the project also conducts further analysis to identify consecutive three-month periods where the campaigns demonstrated an upward trend. This approach enables the team to scrutinize successful periods and examine campaign settings during those periods. By comparing campaign settings during successful periods with current campaign settings, the team can identify any differences and adjust settings accordingly to improve campaign performance.

Overall, this methodology allows for a comprehensive analysis of digital marketing campaigns, identifying trends and pinpointing successful periods to inform future marketing strategies. The project's automation also streamlines the report creation process, saving time and resources and providing the team with valuable insights into campaign performance.

It is worth mentioning that this automation is currently designed to analyze campaign performance on a monthly basis for a period exceeding three months. However, the methodology can be adapted to analyze daily performance for a shorter period than three months. In this case, the project will switch to obtaining trends based on daily performance instead of monthly performance. This flexibility allows the team to adjust the analysis according to their specific needs and obtain valuable insights into campaign performance at different levels of granularity.

**Note:** Please be advised that all data used in the examples and illustrations provided in this conversation are randomly generated and solely for educational and illustrative purposes. Any resemblance to actual data is purely coincidental.

**Comment:** There are explanatory hashtags above each line of code for ease of understanding and being able to follow the process better.

In [1]:
# Import libraries
import pandas as pd 
import numpy as np
import datetime
from sklearn.linear_model import LinearRegression

In [2]:
# Select the excel files 
Purchases = pd.read_excel (r'C:\Users\ASpyrou\Downloads\final\Purchases_Data.xlsx') 
Leads = pd.read_excel (r'C:\Users\ASpyrou\Downloads\final\Leads_Data.xlsx') 
CampaignsData = pd.read_excel (r'C:\Users\ASpyrou\Downloads\final\Campaigns_Data.xlsx') 

The datasets are displayed below to provide a general idea of their structure. The "CampaignsData" includes monthly data for Cost, Impressions, and Clicks, although some campaigns may not have data for every month due to pausing or non-existance.

In [3]:
# Display the purchases dataset
Purchases.head(5)

,Purchase Date,Lead Date,Campaign ID,Customer ID,Purchase Amount
0,2022-12-25 11:38:01,2022-11-08 10:43:09,50,26,2599.0
1,2023-03-20 11:23:29,2022-11-06 23:07:21,30,47,3556.0
2,2023-02-23 12:27:37,2023-02-21 12:27:37,29,29,4205.0
3,2023-02-28 18:16:23,2023-02-22 21:17:09,21,60,1141.0
4,2023-02-27 12:53:03,2022-12-17 22:34:01,18,92,2927.0


In [4]:
# Display the leads dataset
Leads.head(5)

,Lead Date,Campaign ID,Customer ID
0,2022-10-06 02:59:01,63,2386
1,2022-10-09 13:32:32,63,2413
2,2022-10-05 17:19:01,63,2177
3,2022-10-03 18:50:02,63,2348
4,2022-10-08 02:22:01,63,2375


In [5]:
# Display the campaigns dataset
CampaignsData.head(5)

,Month,Campaign ID,Currency code,Impr.,Clicks,Cost
0,2022-10-01,1,USD,32867,367,479.61
1,2022-10-01,2,USD,5923,434,1740.85
2,2022-11-01,2,USD,17612,927,5366.31
3,2022-12-01,2,USD,8710,564,2529.22
4,2023-01-01,2,USD,22339,1083,6542.10


In [6]:
# Convert the date column to '%Y-%m-%d' format
Purchases['Lead Date'] = pd.to_datetime(Purchases['Lead Date'], format='%Y-%m-%d')
Leads['Lead Date'] = pd.to_datetime(Leads['Lead Date'], format='%Y-%m-%d')
CampaignsData['Month'] = pd.to_datetime(CampaignsData['Month'], format='%Y-%m-%d')

# Create a new column called 'year_month' in the dataframe by extracting the year and month from the date column
Leads['year_month'] = Leads['Lead Date'].dt.strftime('%Y-%m')
Purchases['year_month'] = Purchases['Lead Date'].dt.strftime('%Y-%m')
CampaignsData['year_month'] = CampaignsData['Month'].dt.strftime('%Y-%m')

Calculating the CPL of the campaign per month 

In [7]:
# Group the rows in the dataframe by the 'year_month' column and count the number of unique 'id' values for each group
Leadcount = Leads.groupby(['year_month', 'Campaign ID']).size().reset_index(name='Lead_Count')

# Merge the Lead counts on our report based on two columns
CampaignsData = CampaignsData.merge(Leadcount,on=['Campaign ID', 'year_month'],how='left') 

# CPL Calculation
CampaignsData['CPL'] =  CampaignsData['Cost'] / CampaignsData['Lead_Count']

Count the number of Purchases of the campaign per month 

In [8]:
# Creating a New Dataframe that shows the count of purchases per Campaign ID it also resets the index and give the calculated column a name
Purchasescount = Purchases.groupby(['Campaign ID', 'year_month']).size().reset_index(name="Purchases_Count")

# Vlookup the purchase counts on our report based on three columns
CampaignsData = CampaignsData.merge(Purchasescount,on=['Campaign ID', 'year_month'],how='left') 

Sum the Purchases of the campaign per month

In [9]:
# Calculating the campaign purchases per month
PurchasesAmount = Purchases.groupby(['Campaign ID', 'year_month']).agg({'Purchase Amount': 'sum'}).reset_index()

# Renaming the column to 'Purchase_Sum'
PurchasesAmount.rename(columns = {'Purchase Amount':'Purchase_Sum'}, inplace = True)

# Merge the 'Purchase_Sum' on our report based on three columns
CampaignsData = CampaignsData.merge(PurchasesAmount,on=['Campaign ID', 'year_month'],how='left')

Filling in the months missing from each campaign and adding 0 values in all metrics

In [10]:
# Get the unique year_month values from the dataframe
year_months = sorted(CampaignsData['year_month'].unique())

# Create a new dataframe with all possible combinations of Campaign ID and year_month
df_all = pd.DataFrame([(cid, ym) for cid in CampaignsData['Campaign ID'].unique() for ym in year_months], columns=['Campaign ID', 'year_month'])

# Merge the new dataframe with your original dataframe using a left join
CampaignsData = pd.merge(df_all, CampaignsData, on=['Campaign ID', 'year_month'], how='left')

# Fill in the missing values with 0
CampaignsData.fillna(0, inplace=True)

# Drop columns 'Month' and 'Currency code'
Monthly_Campaigns_Data = CampaignsData.drop(['Month', 'Currency code'], axis=1).copy()

Metrics Calculation

In [11]:
# CTR calculation
Monthly_Campaigns_Data['CTR_%'] =  Monthly_Campaigns_Data['Clicks'] / Monthly_Campaigns_Data['Impr.']

# Click to lead calculation
Monthly_Campaigns_Data['Click_to_Lead_%'] =  Monthly_Campaigns_Data['Lead_Count'] / Monthly_Campaigns_Data['Clicks']

# Click to Purchase calculation
Monthly_Campaigns_Data['Click_to_Purchase_%'] =  Monthly_Campaigns_Data['Purchases_Count'] / Monthly_Campaigns_Data['Clicks']

# Lead to Purchase % calculation
Monthly_Campaigns_Data['Lead_to_Purchase_%'] =  Monthly_Campaigns_Data['Purchases_Count'] / Monthly_Campaigns_Data['Lead_Count']

# Cost per Purchase calculation
Monthly_Campaigns_Data['Cost_per_Purchase'] =  Monthly_Campaigns_Data['Cost'] / Monthly_Campaigns_Data['Purchases_Count']

# ROAS calculation
Monthly_Campaigns_Data['ROAS'] =  Monthly_Campaigns_Data['Purchase_Sum'] / Monthly_Campaigns_Data['Cost']

# Replace inf values with NaN values
Monthly_Campaigns_Data = Monthly_Campaigns_Data.replace([np.inf, -np.inf], np.nan)

# Fill NaN values of 'ROAS' with the values from 'Cost'
Monthly_Campaigns_Data['Cost_per_Purchase'].fillna(Monthly_Campaigns_Data['Cost'], inplace=True)

# Replace the values of 'ROAS' with the values of 'Cost' where the values of 'Purchase Sum' are 0
Monthly_Campaigns_Data.loc[Monthly_Campaigns_Data['Purchase_Sum'] == 0, 'ROAS'] = -Monthly_Campaigns_Data['Cost']
Monthly_Campaigns_Data.loc[(Monthly_Campaigns_Data['Cost'] == 0) & (Monthly_Campaigns_Data['Purchase_Sum'] == 0) , 'ROAS'] = Monthly_Campaigns_Data['Cost']

# Fill NaN values of 'ROAS' with the values from 'Cost'
Monthly_Campaigns_Data['Lead_to_Purchase_%'].fillna(0, inplace=True)

Below thresholds for the three key metrics of: ROAS, CPA, and conversion rate are defined. It should be noted that these thresholds serve a demonstrative purpose only. In actual scenarios, CPA thresholds will vary based on the nature of the business. A thorough analysis of customer revenues is necessary to determine the average income per customer and set CPA thresholds that would allow for customer acquisition within budget constraints. 

- ROAS = [1.0, 2.0, 3.0]
- CPA = [900, 600, 500]
- Conversion rate = [0.03, 0.1, 0.2]

Once the thresholds for each metric have been defined in the code, a function is created to assign scores to each of the metrics based on whether they meet the defined thresholds. For instance, if the CPA is less than or equal to the first threshold, the function will assign a score of 1 to that metric. If the CPA is greater than the first threshold, the function will assign a score of 0 to that metric. However, the calculation is inversed for the other two metrics (ROAS and conversion rate). For example, if ROAS is greater than the first threshold, the function assigns a score of 1; otherwise, it assigns a score of 0.

Now suppose we have the following data:
- CPA = 700
- ROAS = 2.3
- Conversion rate = 0.06

Using the functions defined in the code and the thresholds set above, the following scores will be assigned:

    ROAS score: 2 (since ROAS meets the second and third thresholds)
    CPA score: 1 (since the CPA meets the first threshold)
    Conversion rate score: 1 (since the conversion rate meets the first threshold)

These scores are then added together to calculate the overall score, which has a maximum value of 9. The four categories based on the overall score range are:
- "Poor" (0-3)
- "Fair" (4-5)
- "Good" (6-7)
- "Excellent" (8-9)

Based on the example set above the overall score will be 4. This falls within the "Fair" category and will be assigned a performance rating of "Fair".

Categorizing the key metrics into one overall score and assigning it to a category between poor to excellent makes it easier for the viewer to quickly identify good and bad performing campaigns without exerting much effort. Additionally, campaigns that have a fair performance can also be easily identified, allowing the viewer to take appropriate actions. This simplification of data presentation allows for more efficient decision-making and potentially leads to better campaign performance. Moreover, since the data points for each metric are more or less on the same level, linear regression can be used to analyze the trend and determine its direction.

In [12]:
# Define thresholds for each metric
roas_thresholds = [1.0, 2.0, 3.0]
cpa_thresholds = [900, 600, 500]
conversion_rate_thresholds = [0.03, 0.1, 0.2]

# Define the ranges for each category
poor_range = range(0, 4)
fair_range = range(4, 6)
good_range = range(6, 8)
excellent_range = range(8, 10)

# Create a function to assign a score for a given metric and threshold
def score_metric(metric, threshold):
    if metric >= threshold:
        return 1
    else:
        return 0
    
# Create a function to assign a score for a given CPA and threshold
def cpa_score_metric(metric, threshold):
    if metric == 0:
        return 0
    else:
        if metric <= threshold:
            return 1
        else:
            return 0

# Create a function to assign a category for a given overall score
def assign_category(score):
    if score in poor_range:
        return 'Poor'
    elif score in fair_range:
        return 'Fair'
    elif score in good_range:
        return 'Good'
    elif score in excellent_range:
        return 'Excellent'
    
# Create a new dataframe with scores for each metric and overall score
Monthly_Campaigns_Data['ROAS_Score'] = Monthly_Campaigns_Data['ROAS'].apply(lambda x: score_metric(x, roas_thresholds[0]) + 
                                            score_metric(x, roas_thresholds[1]) + 
                                            score_metric(x, roas_thresholds[2]))
Monthly_Campaigns_Data['Cost_per_Purchase_Score'] = Monthly_Campaigns_Data['Cost_per_Purchase'].apply(lambda x: cpa_score_metric(x, cpa_thresholds[0]) + 
                                          cpa_score_metric(x, cpa_thresholds[1]) + 
                                          cpa_score_metric(x, cpa_thresholds[2]))
Monthly_Campaigns_Data['Lead_to_Purchase_Score'] = Monthly_Campaigns_Data['Lead_to_Purchase_%'].apply(lambda x: score_metric(x, conversion_rate_thresholds[0]) + 
                                                                    score_metric(x, conversion_rate_thresholds[1]) + 
                                                                    score_metric(x, conversion_rate_thresholds[2]))

# Adding all the score into an Overall Score
Monthly_Campaigns_Data['Overall_Score'] = Monthly_Campaigns_Data['ROAS_Score'] + Monthly_Campaigns_Data['Cost_per_Purchase_Score'] + Monthly_Campaigns_Data['Lead_to_Purchase_Score']

# Assign a category for the overall score
Monthly_Campaigns_Data['Performance'] = Monthly_Campaigns_Data['Overall_Score'].apply(assign_category)    

The code below performs several data processing steps and applies linear regression analysis to determine the trend for four different campaign performance metrics: overall score, ROAS score, cost per purchase score, and lead to purchase score.

For campaigns that do not have data for all months of the period, the code creates additional rows with 0 values for the missing months.

To determine the trend for each performance metric, the code applies linear regression analysis for each campaign and calculates the slope of the linear regression line. The trend direction is then determined based on the slope, with three possible outcomes: upward trend, downward trend, and stable trend.

If the slope for the ROAS score is 0, indicating that ROAS is not positive on any month, the code replaces the value of the ROAS Score Trend with the corresponding message "ROAS is not positive on any month".

In [13]:
# Extract the year and month from the 'year-month' field
Monthly_Campaigns_Data['year'] = Monthly_Campaigns_Data['year_month'].str[:4].astype(int)
Monthly_Campaigns_Data['month'] = Monthly_Campaigns_Data['year_month'].str[5:].astype(int)

# Sort the dataset based on 3 columns
Monthly_Campaigns_Data = Monthly_Campaigns_Data.sort_values(['Campaign ID', 'year', 'month'])

# Create a new field for the time in months for each campaign
Monthly_Campaigns_Data['time'] = Monthly_Campaigns_Data.groupby('Campaign ID').apply(lambda x: (x['year'] - x['year'].min()) * 12 + x['month'] - x['month'].min() + 1).values

# Create a new field for the time in months (starting from 0) for each campaign
Monthly_Campaigns_Data['time'] = Monthly_Campaigns_Data.groupby('Campaign ID').apply(
    lambda x: (x['year'] - x['year'].min()) * 12 + x['month'] - x['month'].min() + 1 - x['time'].min()
).values 

# Increment the 'time' column by 1 for each row
Monthly_Campaigns_Data['time'] = Monthly_Campaigns_Data['time'] +1

# Create a dictionary to store the slopes for each campaign
slopes = {}

# Create empty dictionaries to store the final outputs for each field
final_outputs = {}
final_outputs_recent ={}

# List of fields to apply the code on
fields = ['Overall_Score', 'ROAS_Score', 'Cost_per_Purchase_Score', 'Lead_to_Purchase_Score']

# Loop over each field
for field in fields:
    # Loop over each campaign and fit a linear regression model
    for campaign, group in Monthly_Campaigns_Data.groupby('Campaign ID'):
        # Fit a linear regression model to the time and field data for the current campaign group
        model = LinearRegression().fit(group[['time']], group[field])
        # Calculate the slope of the linear regression line
        slope = model.coef_[0]
        # Determine the trend direction based on the slope
        trend = ""
        if slope > 0:
            trend = "Upward trend"
        elif slope < 0:
            trend = "Downward trend"
        else:
            trend = "Stable trend"
        # Store the slope and trend data in a dictionary with the campaign ID as the key
        slopes[campaign] = {f'Slope_{field}': slope, f'{field}_Trend': trend}

    # Convert the slopes dictionary to a pandas dataframe
    df_slopes = pd.DataFrame.from_dict(slopes, orient='index').reset_index().rename(columns={'index': 'Campaign ID'})

    # Pivot the data
    pivoted_data = Monthly_Campaigns_Data.pivot_table(values=field, index='Campaign ID', columns=['year_month']).reset_index()

    # Merge the Slopes and the pivoted data
    final_output = pivoted_data.merge(df_slopes, on=['Campaign ID'], how='left')
    
    # Save the final output dataframe as a separate variable
    final_outputs[field] = final_output

# Access the final output dataframes for each field using the variable name
Overall_score_final_output = final_outputs['Overall_Score']
ROAS_score_final_output = final_outputs['ROAS_Score']
Cost_per_Purchase_score_final_output = final_outputs['Cost_per_Purchase_Score']
Lead_to_Purchase_score_final_output = final_outputs['Lead_to_Purchase_Score']

# Replace the values of ROAS Score Trend if the values of Slope are 0
ROAS_score_final_output.loc[ROAS_score_final_output['Slope_ROAS_Score'] == 0, 'ROAS Score_Trend'] = "ROAS is not positive on any month"

An illustration of the Overall Score's outcomes is provided below. By examining the 'Overall_Score_Trend' feature and each campaign's monthly score, we can clearly observe and validate the campaign's performance throughout the months.

In [14]:
Overall_score_final_output.head(10)

,Campaign ID,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,Slope_Overall_Score,Overall_Score_Trend
0,1,3,0,0,0,0,0,-0.428571,Downward trend
1,2,4,2,3,2,0,0,-0.771429,Downward trend
2,3,3,0,0,0,0,0,-0.428571,Downward trend
3,4,2,0,0,0,0,0,-0.285714,Downward trend
4,5,0,0,0,0,0,0,0.000000,Stable trend
5,6,3,3,0,0,0,0,-0.685714,Downward trend
6,7,0,0,4,0,1,3,0.400000,Upward trend
7,8,3,3,3,0,0,0,-0.771429,Downward trend
8,9,3,4,5,4,0,0,-0.800000,Downward trend
9,10,0,3,1,0,0,1,-0.142857,Downward trend


Expanding on the previous statement, the code below is specifically created to identify the two most recent 3-month periods that exhibit an increasing trend in three distinct metrics, namely Overall Score, ROAS Score, and Cost per Purchase Score. To accomplish this, the function "calculate_slopes" has been implemented, which calculates the slopes for each campaign and each 3-month period for a specified field or metric, and returns the data in the form of a DataFrame. By doing so, we are able to identify the periods that display a positive trend, indicating an increase in performance. This information can be extremely valuable in evaluating the success of a campaign and identifying areas for further improvement.

In [15]:
# Define a function that takes a field as input and returns a DataFrame with the slope data for that field
def calculate_slopes(field):
    slopes = []
    n_periods = 3
    # Group the data by Campaign ID and loop through each group
    for campaign, group in Monthly_Campaigns_Data.groupby('Campaign ID'):
        # Get the maximum time period for the current campaign
        max_time = group['time'].max()
        # Loop through all possible time periods of length n_periods
        for i in range(1, max_time - n_periods + 2):
            start_time = i
            end_time = i + n_periods - 1
            # Select the data for the current time period
            sub_group = group[(group['time'] >= start_time) & (group['time'] <= end_time)]
            # If there is data for the current time period, fit a linear regression model and calculate the slope
            if len(sub_group) > 0:
                model = LinearRegression().fit(sub_group[['time']], sub_group[field])
                slope = model.coef_[0]
                # Determine the trend direction based on the slope
                trend = ""
                if slope > 0:
                    trend = "Upward trend"
                elif slope < 0:
                    trend = "Downward trend"
                else:
                    trend = "Stable trend"
                # Store the slope data in a list
                slopes.append({'Campaign ID': campaign, 'Period': f"{start_time}-{end_time}", 'Slope': slope, 'Trend': trend})
    # Convert the list of slope data to a DataFrame and return it
    return pd.DataFrame(slopes)

# Create an empty dictionary to store the output DataFrames for each field
final_outputs_recent = {}
# Loop through the list of fields and call the calculate_slopes function for each field
for field in fields:
    final_outputs_recent[field] = calculate_slopes(field)

# Create a dictionary to store the filtered dictionaries with the upward trends
final_outputs_recent_filtered = {}

# Filtering only the upward trend 
for key in final_outputs_recent:
    # Filter the current dataframe based on the 'Trend' column
    filtered_df = final_outputs_recent[key][final_outputs_recent[key]['Trend'] == 'Upward trend'].copy()
    
    # Extract the first and second numbers from the 'Period' column as separate columns
    filtered_df[['Period_1', 'Period_2']] = filtered_df['Period'].str.extract('(\d+)-(\d+)')
    
    # Convert the columns to numerical dtype
    filtered_df['Period_1'] = filtered_df['Period_1'].astype(int)
    filtered_df['Period_2'] = filtered_df['Period_2'].astype(int)
    
    # Group the filtered dataframe by campaign ID and sort by 'Campaign ID' and the second number
    grouped = filtered_df.sort_values(['Campaign ID', 'Period_2'], ascending=[True, False]).groupby('Campaign ID')
    
    # Take the top two rows for each group and add to new dictionary
    final_outputs_recent_filtered[key] = grouped.head(2).copy()
    
    
# Creation of a mapping dataframe where its values will replace the period numbers
Year_Month_Period_Mapping = Monthly_Campaigns_Data[['year_month', 'time']].drop_duplicates()

# Converting the "time" column to a string type
Year_Month_Period_Mapping["time"] = Year_Month_Period_Mapping["time"].astype(str)

# Iterate over each dataframe in the dictionary
for df_name, df in final_outputs_recent_filtered.items():
    # Split the "Period" column into two separate columns
    df[["start_period", "end_period"]] = df["Period"].str.split("-", expand=True)
    # Merge with the "Year_Month_Period_Mapping" dataframe to get the start and end year/months
    df = df.merge(Year_Month_Period_Mapping, left_on="start_period", right_on="time")
    df = df.merge(Year_Month_Period_Mapping, left_on="end_period", right_on="time")
    # Create a new "Period" column by combining the start and end year/months
    df["Period"] = df["year_month_x"] + " to " + df["year_month_y"]
    # Drop unnecessary columns
    df = df.drop(["start_period", "end_period", "year_month_x", "year_month_y", "time_x", "time_y"], axis=1)
    # Replace the dataframe in the dictionary with the updated dataframe
    final_outputs_recent_filtered[df_name] = df

# Access the final output dataframes for each field using the variable name and sorting on the latest period first
Overall_score_final_output_periods = final_outputs_recent_filtered['Overall_Score'].sort_values(['Campaign ID','Period'], ascending=False)
ROAS_score_final_output_periods = final_outputs_recent_filtered['ROAS_Score'].sort_values(['Campaign ID','Period'], ascending=False)
Cost_per_Purchase_score_final_output_periods = final_outputs_recent_filtered['Cost_per_Purchase_Score'].sort_values(['Campaign ID','Period'], ascending=False)
Lead_to_Purchase_score_final_output_periods = final_outputs_recent_filtered['Lead_to_Purchase_Score'].sort_values(['Campaign ID','Period'], ascending=False)


def merge_and_pivot(df1, df2, on_col, value_col, period_col):
    # Merge the dataframes on the specified column
    merged_df = pd.merge(df1, df2, on=on_col, how='left')
    
    # Use pivot_table to create two new columns based on the period column
    pivot_df = merged_df.pivot_table(index=on_col, columns=merged_df.groupby(on_col).cumcount(), values=period_col, aggfunc='first')
    
    # Rename the columns to something more descriptive
    pivot_df.columns = [f'Recent Period {i}' for i in range(len(pivot_df.columns))]
    
    # Merge the pivoted dataframe with the original dataframe to include the new columns
    output_df = pd.merge(df1, pivot_df, on=on_col, how='left')
    
    return output_df

# Call the merge_and_pivot function for each score and assign the output to new dataframes
Overall_score_final_output = merge_and_pivot(Overall_score_final_output, Overall_score_final_output_periods, 'Campaign ID', 'Overall Score', 'Period')
ROAS_score_final_output = merge_and_pivot(ROAS_score_final_output, ROAS_score_final_output_periods, 'Campaign ID', 'ROAS_Score Score', 'Period')
Cost_per_Purchase_score_final_output = merge_and_pivot(Cost_per_Purchase_score_final_output, Cost_per_Purchase_score_final_output_periods, 'Campaign ID', 'Cost_per_Purchase_Score', 'Period')
Lead_to_Purchase_score_final_output = merge_and_pivot(Lead_to_Purchase_score_final_output, Lead_to_Purchase_score_final_output_periods, 'Campaign ID', 'Lead_to_Purchase_Score', 'Period')


The final outcome of our DataFrame can now be seen, displaying two new columns that indicate the two most recent 3-month periods with an upward trend. For instance, the campaign listed in row 8 has a score across four months, showing an overall downward trend. However, we can clearly see an upward trend during the months of October 22 to December 22. This enables us to revisit that time period and analyze the measures taken during that period, so we can implement any successful changes that were made.

In [16]:
Overall_score_final_output.head(10)

,Campaign ID,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,Slope_Overall_Score,Overall_Score_Trend,Recent Period 0,Recent Period 1
0,1,3,0,0,0,0,0,-0.428571,Downward trend,NaN,NaN
1,2,4,2,3,2,0,0,-0.771429,Downward trend,NaN,NaN
2,3,3,0,0,0,0,0,-0.428571,Downward trend,NaN,NaN
3,4,2,0,0,0,0,0,-0.285714,Downward trend,NaN,NaN
4,5,0,0,0,0,0,0,0.000000,Stable trend,NaN,NaN
5,6,3,3,0,0,0,0,-0.685714,Downward trend,NaN,NaN
6,7,0,0,4,0,1,3,0.400000,Upward trend,2023-01 to 2023-03,2022-10 to 2022-12
7,8,3,3,3,0,0,0,-0.771429,Downward trend,NaN,NaN
8,9,3,4,5,4,0,0,-0.800000,Downward trend,2022-10 to 2022-12,NaN
9,10,0,3,1,0,0,1,-0.142857,Downward trend,2023-01 to 2023-03,2022-10 to 2022-12


Having established the overall monthly trend for the campaign, and identified two recent periods during which the campaign exhibited a good performance, we will now proceed to calculate the campaign's overall performance for the entire period.

In [17]:
# Calculating the totals per campaign
Campaign_Summary = Monthly_Campaigns_Data.groupby('Campaign ID').agg({'Cost': 'sum', 'Impr.': 'sum','Clicks': 'sum', 'Lead_Count': 'sum',
                                                                      'Purchases_Count': 'sum', 'Purchase_Sum': 'sum',
                                                                     }).reset_index()

# CTR calculation
Campaign_Summary['CTR_%'] =  Campaign_Summary['Clicks'] / Campaign_Summary['Impr.']

# Click to lead calculation
Campaign_Summary['Click_to_Lead_%'] =  Campaign_Summary['Lead_Count'] / Campaign_Summary['Clicks']

# Click to Purchase calculation
Campaign_Summary['Click_to_Purchase_%'] =  Campaign_Summary['Purchases_Count'] / Campaign_Summary['Clicks']

# Lead to Purchase % calculation
Campaign_Summary['Lead_to_Purchase_%'] =  Campaign_Summary['Purchases_Count'] / Campaign_Summary['Lead_Count']

# Cost per Purchase calculation
Campaign_Summary['Cost_per_Purchase'] =  Campaign_Summary['Cost'] / Campaign_Summary['Purchases_Count']

# ROAS calculation
Campaign_Summary['ROAS'] =  Campaign_Summary['Purchase_Sum'] / Campaign_Summary['Cost']

# Replace inf values with NaN values
Campaign_Summary = Campaign_Summary.replace([np.inf, -np.inf], np.nan)

# Fill NaN values of 'ROAS' with the values from 'Cost'
Campaign_Summary['Cost_per_Purchase'].fillna(Campaign_Summary['Cost'], inplace=True)

# Replace the values of 'ROAS' with the values of 'Cost' where the values of 'Purchase_Sum' are 0
Campaign_Summary.loc[Campaign_Summary['Purchase_Sum'] == 0, 'ROAS'] = - Campaign_Summary['Cost']

# Fill NaN values of 'ROAS' with the values from 'Cost'
Campaign_Summary['Lead_to_Purchase_%'].fillna(0, inplace=True)


In the following section, we will calculate the overall score for the campaign, as well as the scores for each of its key metrics, this will allow us to gain a better understanding of the campaign's overall performance. 

In [18]:
# Create a new dataframe with scores for each metric and overall score
Campaign_Summary['ROAS_Score'] = Campaign_Summary['ROAS'].apply(lambda x: score_metric(x, roas_thresholds[0]) + 
                                            score_metric(x, roas_thresholds[1]) + 
                                            score_metric(x, roas_thresholds[2]))
Campaign_Summary['Cost_per_Purchase_Score'] = Campaign_Summary['Cost_per_Purchase'].apply(lambda x: cpa_score_metric(x, cpa_thresholds[0]) + 
                                          cpa_score_metric(x, cpa_thresholds[1]) + 
                                          cpa_score_metric(x, cpa_thresholds[2]))
Campaign_Summary['Lead_to_Purchase_Score'] = Campaign_Summary['Lead_to_Purchase_%'].apply(lambda x: score_metric(x, conversion_rate_thresholds[0]) + 
                                                                    score_metric(x, conversion_rate_thresholds[1]) + 
                                                                    score_metric(x, conversion_rate_thresholds[2]))

# Adding all the score into an Overall Score
Campaign_Summary['AllTime_Score'] = Campaign_Summary['ROAS_Score'] + Campaign_Summary['Cost_per_Purchase_Score'] + Campaign_Summary['Lead_to_Purchase_Score']

# Assign a category for the Overall Score
Campaign_Summary['AllTime_Performance'] = Campaign_Summary['AllTime_Score'].apply(assign_category)

Now that we have collected all the necessary data, we can proceed to compile it into a final table. This table will include the campaign's overall score and a comprehensive analysis of its performance over the entire period, as well as the trend exhibited by the campaign over the months. Additionally, we will include the two most recent periods during which the campaign had good performance in the table for reference.

In [19]:
# Renaming the columns
Overall_score_final_output.rename(columns = {'Recent Period 0':'Overall_Score_Recent_Period_0', 
                                             'Recent Period 1':'Overall_Score_Recent_Period_1', 
                                             'Overall_Score_Trend':'Overall_Score_Monthly_Trend'}, inplace = True)

ROAS_score_final_output.rename(columns = {'Recent Period 0':'ROAS_Recent_Period_0', 
                                          'Recent Period 1':'ROAS_Recent_Period_1', 
                                          'ROAS_Score_Trend':'ROAS_Monthly_Trend'}, inplace = True)

Cost_per_Purchase_score_final_output.rename(columns = {'Recent Period 0':'Cost_per_Purchase_Recent_Period_0', 
                                                  'Recent Period 1':'Cost_per_Purchase_Recent_Period_1', 
                                                  'Cost_per_Purchase_Score_Trend':'Cost_per_Purchase_Monthly_Trend'}, inplace = True)

Lead_to_Purchase_score_final_output.rename(columns = {'Recent Period 0':'Lead_to_Purchase_Recent_Period_0', 
                                                 'Recent Period 1':'Lead_to_Purchase_Recent_Period_1', 
                                                 'Lead_to_Purchase_Score_Trend':'Lead_to_Purchase_Monthly_Trend'}, inplace = True)

# Merge the two dataframes on the Campaign ID column and keep only the columns needed from the second dataframe
Campaign_Summary = pd.merge(Campaign_Summary, Overall_score_final_output[['Campaign ID', 'Overall_Score_Monthly_Trend',
                                                                          'Overall_Score_Recent_Period_0','Overall_Score_Recent_Period_1']],
                            on='Campaign ID', how='left')

Campaign_Summary = pd.merge(Campaign_Summary, ROAS_score_final_output[['Campaign ID', 'ROAS_Monthly_Trend',
                                                                          'ROAS_Recent_Period_0','ROAS_Recent_Period_1']],
                            on='Campaign ID', how='left')

Campaign_Summary = pd.merge(Campaign_Summary, Cost_per_Purchase_score_final_output[['Campaign ID', 'Cost_per_Purchase_Monthly_Trend',
                                                                          'Cost_per_Purchase_Recent_Period_0','Cost_per_Purchase_Recent_Period_1']],
                            on='Campaign ID', how='left')

Campaign_Summary = pd.merge(Campaign_Summary, Lead_to_Purchase_score_final_output[['Campaign ID', 'Lead_to_Purchase_Monthly_Trend',
                                                                          'Lead_to_Purchase_Recent_Period_0','Lead_to_Purchase_Recent_Period_1']],
                            on='Campaign ID', how='left')



Now, let's print the final table. The data can be exported to an Excel file, allowing the application of various filters and conditional formatting to split the campaigns into different categories for better analysis. For example, we could filter campaigns with a "Poor" rating in their "AllTime_Performance" field and check if there were any recent periods with good performance. We can make appropriate changes similar to those made during the good performing periods. If there are no good performing periods and the campaign has been running for a long time, pausing it might be an option worth considering. Additionally, we can identify campaigns with an "Excellent" rating in their "AllTime_Performance" field and consider increasing their budget, if feasible.

In [20]:
# Replace NaN values with "-"
Campaign_Summary.fillna('-', inplace=True)

# Set the maximum number of columns to display
pd.options.display.max_columns = 30

# Display the campaign analysis final dataset 
Campaign_Summary.head(10)

,Campaign ID,Cost,Impr.,Clicks,Lead_Count,Purchases_Count,Purchase_Sum,CTR_%,Click_to_Lead_%,Click_to_Purchase_%,Lead_to_Purchase_%,Cost_per_Purchase,ROAS,ROAS_Score,Cost_per_Purchase_Score,Lead_to_Purchase_Score,AllTime_Score,AllTime_Performance,Overall_Score_Monthly_Trend,Overall_Score_Recent_Period_0,Overall_Score_Recent_Period_1,ROAS_Monthly_Trend,ROAS_Recent_Period_0,ROAS_Recent_Period_1,Cost_per_Purchase_Monthly_Trend,Cost_per_Purchase_Recent_Period_0,Cost_per_Purchase_Recent_Period_1,Lead_to_Purchase_Monthly_Trend,Lead_to_Purchase_Recent_Period_0,Lead_to_Purchase_Recent_Period_1
0,1,479.61,32867.0,367.0,4.0,0.0,0.00,0.011166,0.010899,0.000000,0.000000,479.610000,-479.610000,0,3,0,3,Poor,Downward trend,-,-,Stable trend,-,-,Downward trend,-,-,Stable trend,-,-
1,2,32753.35,110424.0,6185.0,187.0,13.0,11252.60,0.056011,0.030234,0.002102,0.069519,2519.488462,0.343556,0,0,1,1,Poor,Downward trend,-,-,Stable trend,-,-,Downward trend,2022-10 to 2022-12,-,Downward trend,-,-
2,3,109.01,316.0,154.0,0.0,0.0,0.00,0.487342,0.000000,0.000000,0.000000,109.010000,-109.010000,0,3,0,3,Poor,Downward trend,-,-,Stable trend,-,-,Downward trend,-,-,Stable trend,-,-
3,4,11337.55,46704.0,3067.0,82.0,0.0,0.00,0.065669,0.026736,0.000000,0.000000,11337.550000,-11337.550000,0,0,0,0,Poor,Downward trend,-,-,Stable trend,-,-,Downward trend,-,-,Stable trend,-,-
4,5,5186.18,12017.0,762.0,21.0,0.0,0.00,0.063410,0.027559,0.000000,0.000000,5186.180000,-5186.180000,0,0,0,0,Poor,Stable trend,-,-,Stable trend,-,-,Stable trend,-,-,Stable trend,-,-
5,6,493.72,16635.0,392.0,1.0,0.0,0.00,0.023565,0.002551,0.000000,0.000000,493.720000,-493.720000,0,3,0,3,Poor,Downward trend,-,-,Stable trend,-,-,Downward trend,-,-,Stable trend,-,-
6,7,8204.20,38218.0,2533.0,60.0,4.0,2793.17,0.066278,0.023687,0.001579,0.066667,2051.050000,0.340456,0,0,1,1,Poor,Upward trend,2023-01 to 2023-03,2022-10 to 2022-12,Downward trend,2022-10 to 2022-12,-,Upward trend,2023-01 to 2023-03,2022-10 to 2022-12,Upward trend,2023-01 to 2023-03,2022-10 to 2022-12
7,8,1208.95,93051.0,1331.0,1.0,0.0,0.00,0.014304,0.000751,0.000000,0.000000,1208.950000,-1208.950000,0,0,0,0,Poor,Downward trend,-,-,Stable trend,-,-,Downward trend,-,-,Stable trend,-,-
8,9,2727.25,133981.0,4231.0,193.0,7.0,2590.11,0.031579,0.045616,0.001654,0.036269,389.607143,0.949715,0,3,1,4,Fair,Downward trend,2022-10 to 2022-12,-,Downward trend,2022-10 to 2022-12,-,Downward trend,2022-11 to 2023-01,2022-10 to 2022-12,Downward trend,2022-10 to 2022-12,-
9,10,4759.51,9443.0,1515.0,29.0,1.0,357.50,0.160436,0.019142,0.000660,0.034483,4759.510000,0.075113,0,0,1,1,Poor,Downward trend,2023-01 to 2023-03,2022-10 to 2022-12,Stable trend,-,-,Upward trend,2023-01 to 2023-03,2022-10 to 2022-12,Downward trend,-,-
